## Leveraging Dask for timeseries sampling

### spin up a dask client 


### multithreading and yt

Note that `yt` is not gauranteed to be thread safe. In practice, there are some situations where it **might** be and for this particular dataset, it actually is as long as logging is disabled (which is enforced within `add_to_viewer` when `use_dask=True`. 

In [ ]:
from dask.distributed import Client 

In [ ]:
c = Client(n_workers=5, threads_per_worker=5)

In [ ]:
c

In [ ]:
import napari 
from yt_napari import timeseries

## Delayed image stacks 

### Slices

In [ ]:
%%capture
slc = timeseries.Slice(("enzo", "Density"), "x")
timeseries.add_to_viewer(v, s, file_pattern=file_pattern, load_as_stack=True, 
                         use_dask=True, 
                         contrast_limits=(-1, 2))

### Regions

In [ ]:
%%capture
reg = timeseries.Region(("enzo", "Density"), resolution=(100, 100, 100))
timeseries.add_to_viewer(v, reg, file_pattern=file_pattern, load_as_stack=True, 
                         use_dask=True, 
                         contrast_limits=(-1, 2))

## Using dask, returning in-memory image array 

We can also use dask to simply distribute the selection but still return a fully in-memory array. This works best for slices, where you **probably** can safely fit all those slices in memory. 

In [ ]:
%%time
slice = timeseries.Slice(("enzo", "Density"), "x", resolution=(1600, 1600))
timeseries.add_to_viewer(v, slice, file_pattern=file_pattern, load_as_stack=True, 
                         use_dask=True, 
                         return_delayed = False,
                         contrast_limits=(-1, 2))